# sentences

## Set Up

Import modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
import gensim
import re

import bookcave
import train_embeddings

Load data.

In [ ]:
inputs, Y, categories, levels, \
book_ids, books_df, _, _, _ = bookcave.get_data({'text', 'images'},
                                                text_input='filename',
                                                return_meta=True)

## Example

View the labels for an example document.

In [ ]:
book_ids[20:30]

In [ ]:
[(Y[20][i], levels[i][j]) for i, j in enumerate(Y[20])]

View a snippet of the text for this book.

In [ ]:
with open(texts[42], 'r', encoding='utf-8') as fd:
    text = fd.read()

In [ ]:
text[:200]

Split the text into lines.

In [ ]:
lines = text.split('\n')

In [ ]:
len(lines)

In [ ]:
lines[:13]

Split the lines into sentences.

In [ ]:
def get_sentences(lines, line_endings=None):
    for line in lines:
        for sentence in nltk.sent_tokenize(line):
            yield sentence

In [ ]:
sentences = np.array(list(get_sentences(lines)))

In [ ]:
len(sentences)

In [ ]:
sentences[:20]

In [ ]:
sentences[114]

In [ ]:
normalized_example = re.sub("[“”]", '"', sentences[114])
normalized_example

In [ ]:
nltk.sent_tokenize(normalized_example)

Tokenize the sentences.

In [ ]:
def get_processed_sentences(sentences):
    # Sentence ID, or the ID number (index) of the original sentence in `sentences`.
    for sid, sentence in enumerate(sentences):
        tokens = list(gensim.utils.simple_preprocess(sentence))
#         tokens = list(gensim.utils.tokenize(sentence, lowercase=True))
        if len(tokens) > 0:
            yield (sid, tokens)

In [ ]:
processed_sentences = list(get_processed_sentences(sentences))

In [ ]:
len(processed_sentences)

In [ ]:
processed_sentences[:23]

Load a pre-trained `doc2vec` model.

In [ ]:
doc_model = train_embeddings.load_doc_model('docmodel_sentence_50d_8w_2min_20e.model')

Convert the sentences of the example document into fixed-length vectors.

In [ ]:
doc_vectors = np.zeros((len(processed_sentences), doc_model.vector_size))
for i, (sid, processed_sentence) in enumerate(processed_sentences):
    doc_vectors[i] = doc_model.infer_vector(processed_sentence)

Visualize the sentence embeddings using t-SNE.

In [ ]:
sklearn.metrics.pairwise.PAIRWISE_DISTANCE_FUNCTIONS

In [ ]:
doc_embedded = TSNE(n_components=2, metric='euclidean').fit_transform(doc_vectors)
doc_embedded.shape

In [ ]:
plt.scatter(doc_embedded[:, 0], doc_embedded[:, 1])
plt.show()

View a clustering of the t-SNE results.

In [ ]:
colors = ['red', 'blue', 'green', 'yellow', 'magenta', 'cyan', 'brown']

In [ ]:
def plot_clusters(points, labels):
    c = [colors[label] for label in labels]
    plt.scatter(points[:, 0], points[:, 1], c=c)
    plt.show()

In [ ]:
clustering_embedded_single = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='single').fit(doc_embedded)
plot_clusters(doc_embedded, clustering_embedded_single.labels_)

In [ ]:
clustering_embedded_complete = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='complete').fit(doc_embedded)
plot_clusters(doc_embedded, clustering_embedded_complete.labels_)

In [ ]:
clustering_embedded_average = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='average').fit(doc_embedded)
plot_clusters(doc_embedded, clustering_embedded_average.labels_)

In [ ]:
clustering_embedded_ward = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward').fit(doc_embedded)
plot_clusters(doc_embedded, clustering_embedded_ward.labels_)

Now fit the multi-dimensional vectors using a clustering algorithm.

In [ ]:
clustering_full_3_single = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='single').fit(doc_vectors)
plot_clusters(doc_embedded, clustering_full_3_single.labels_)

In [ ]:
clustering_full_3_complete = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='complete').fit(doc_vectors)
plot_clusters(doc_embedded, clustering_full_3_complete.labels_)

In [ ]:
clustering_full_3_average = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='average').fit(doc_vectors)
plot_clusters(doc_embedded, clustering_full_3_average.labels_)

In [ ]:
clustering_full_3_ward = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward').fit(doc_vectors)
plot_clusters(doc_embedded, clustering_full_3_ward.labels_)

Try 4.

In [ ]:
clustering_full_4_single = AgglomerativeClustering(n_clusters=4, affinity='cosine', linkage='single').fit(doc_vectors)
plot_clusters(doc_embedded, clustering_full_4_single.labels_)

In [ ]:
clustering_full_4_complete = AgglomerativeClustering(n_clusters=4, affinity='cosine', linkage='complete').fit(doc_vectors)
plot_clusters(doc_embedded, clustering_full_4_complete.labels_)

In [ ]:
clustering_full_4_average = AgglomerativeClustering(n_clusters=4, affinity='cosine', linkage='average').fit(doc_vectors)
plot_clusters(doc_embedded, clustering_full_4_average.labels_)

What kinds of sentence clusters are we seeing above?

In [ ]:
def print_sample_sentences(clustering, size=10):
    for cluster_index in range(clustering.n_clusters):
        print('Cluster {:d} (`{}`):'.format(cluster_index + 1, colors[cluster_index]))
        # Processed sentence IDs (indices).
        psids = np.array([i for i, label in enumerate(clustering.labels_) if label == cluster_index])
        sids = np.array([processed_sentences[psid][0] for psid in psids])
        for sentence in np.random.choice(sentences[sids], size=size, replace=False):
            print('  {}'.format(sentence))

In [ ]:
print_sample_sentences(clustering_full_4, size=5)

Try 5.

In [ ]:
clustering_full_5 = AgglomerativeClustering(n_clusters=5, linkage='ward').fit(doc_vectors)
plot_clusters(doc_embedded, clustering_full_5.labels_)

In [ ]:
print_sample_sentences(clustering_full_5, size=5)